In [ ]:
import numpy as np
import torch
from NeuralNet import NeuralNetwork as Network
import time
from Utilities import Utilities as Utils
from IPython.display import clear_output

In [ ]:
Filters = 128
Layers = 20
HistoryDepth = 8
KernalSize = 3

model = Network(Filters, HistoryDepth + 1, Layers, kernal_size=KernalSize)

model.load_state_dict(torch.load('../../Models/HumanModels/128f20l/4.pt'))
model.eval()

In [ ]:
board = np.zeros((HistoryDepth + 1, 15, 15), dtype=bool)

In [ ]:
player = 1
while (True):
    if player % 2:
        modelInput = torch.from_numpy(board.astype(np.float32)).unsqueeze(0)
        with torch.no_grad():
            output = model(modelInput)
        policyOut, valueOut = output
        policyOut = torch.nn.functional.softmax(policyOut, dim=1)
        policyOut = np.array(policyOut)
        policyOut = policyOut.reshape(15, 15)
        policyOut *= ~board[1]
        policyOut *= ~board[HistoryDepth // 2 + 1]
        policyOut = policyOut.flatten()

        valueOut = valueOut[0].item()

        index = policyOut.argmax()
        x, y = int(index // 15), int(index % 15)
        board = Utils.makeMove(board, x, y)
        clear_output(wait=True)
        print(x,y)
        print(f'Evaluation: {valueOut}')
        print(Utils.sliceGamestate(board, 0))
        time.sleep(1)
    else:
        x = int(input("X:"))
        y = int(input("Y:"))

        board = Utils.makeMove(board, x, y)
        clear_output(wait=True)
        print(Utils.sliceGamestate(board, 0))
        time.sleep(1)
    player += 1